In [218]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time

In [ ]:
"//div[contains(@class, 'webix_dtable')]"
"//div[@class='webix_ss_body']"
"webix_column align-l webix_first"

In [186]:
# Function to click on a box within the table
def click_on_box():
    # Locate and click on the box within the table
    box = driver.find_element("xpath", "//div[contains(@class, 'webix_ss_center_scroll')]")
    box.click()

In [41]:
def scroll_table_with_cursor(driver, amount):
    table = driver.find_element(By.CLASS_NAME, "webix_ss_center_scroll")
    # Move the cursor to the table element
    ActionChains(driver).move_to_element(table).perform()
    
    # Scroll the table using the scroll wheel
    for _ in range(abs(amount)):
        if amount > 0:
            ActionChains(driver).send_keys(Keys.ARROW_DOWN).perform()
        else:
            ActionChains(driver).send_keys(Keys.ARROW_UP).perform()

In [234]:
# Setup options for the headless browser
options = Options()
options.add_argument("--headless")

# Initialize the browser
driver = webdriver.Firefox(options=options)



# Load the webpage
print("Opening Window")
driver.get("https://www.rotowire.com/betting/nba/archive.php")
driver.maximize_window()
driver.execute_script("window.scrollTo(0, 400)")
time.sleep(5)  # Wait for the page to load (You can replace this with more reliable wait conditions)
click_on_box()

table = driver.find_element("xpath", "//div[@class='webix_ss_body']")
table_text = pd.DataFrame(columns=["Game", "Date", "Year", "Score", "Over/Under", "Home Line Spread"])
for i in range(835):  # Adjust the number of times to scroll based on your specific case
    
    if (i+1) % 25 == 0:
        print(f"Iteration {i+1} of 835\n {round((i+1)/835 * 100, 1)}% Complete")
    
    text = table.text.split("\n")
    for j, item in enumerate(text):
        if "@" in item:
            index = j
        else:
            break
    
    games, dates, years, scores, ous, hls = [], [], [], [], [], []
    
    games.append(text[0:index+1])
    games = [item for sublist in games for item in sublist]
    
    dates.append(text[index+1:2*(index+1)])
    dates = [item for sublist in dates for item in sublist]
    
    years.append(text[2*(index+1):3*(index+1)])
    years = [item for sublist in years for item in sublist]
    
    scores.append(text[3*(index+1):4*(index+1)])
    scores = [item for sublist in scores for item in sublist]
    
    if len(text) < 6*(index+1):
        print(len(games), len(dates), len(years), len(scores))
        print("Missing Data", print(text))
        ous = np.zeros(len(games))
        hls = text[-len(games):]
    else:
        ous.append(text[4*(index+1):5*(index+1)])
        ous = [item for sublist in ous for item in sublist]
        hls.append(text[5*(index+1):6*(index+1)])
        hls = [item for sublist in hls for item in sublist]
    
            
    data_dict = {
        "Game": games,
        "Date": dates,
        "Year": years,
        "Score": scores,
        "Over/Under": ous,
        "Home Line Spread": hls
    }
    
    table_text = pd.concat([table_text, pd.DataFrame(data_dict)])    
    
    time.sleep(.1)
    scroll_table_with_cursor(driver, 10)
    time.sleep(.1)
    
# Close the browser
driver.quit()

# Process Scraped Data
table_text

Opening Window
Iteration 25 of 835
 3.0% Complete
Iteration 50 of 835
 6.0% Complete
Iteration 75 of 835
 9.0% Complete
Iteration 100 of 835
 12.0% Complete
Iteration 125 of 835
 15.0% Complete
Iteration 150 of 835
 18.0% Complete
Iteration 175 of 835
 21.0% Complete
Iteration 200 of 835
 24.0% Complete
Iteration 225 of 835
 26.9% Complete
Iteration 250 of 835
 29.9% Complete
Iteration 275 of 835
 32.9% Complete
Iteration 300 of 835
 35.9% Complete
Iteration 325 of 835
 38.9% Complete
Iteration 350 of 835
 41.9% Complete
Iteration 375 of 835
 44.9% Complete
Iteration 400 of 835
 47.9% Complete
Iteration 425 of 835
 50.9% Complete
Iteration 450 of 835
 53.9% Complete
Iteration 475 of 835
 56.9% Complete
Iteration 500 of 835
 59.9% Complete
Iteration 525 of 835
 62.9% Complete
11 11 11 11
['LAL @ DAL', 'IND @ MIL', 'NOP @ OKC', 'CHA @ SAS', 'MIN @ DEN', 'MEM @ POR', 'WAS @ SAC', 'LAC @ UTA', 'DET @ IND', 'PHI @ BKN', 'NYK @ HOU', 'Dec 15 7:30 PM', 'Dec 15 8:00 PM', 'Dec 15 8:00 PM', 'Dec

,Game,Date,Year,Score,Over/Under,Home Line Spread
0,BOS @ CLE,Oct 17 8:00 PM,2017,99-102,216.0,-4.5
1,HOU @ GSW,Oct 17 10:30 PM,2017,122-121,231.0,-9.5
2,CHA @ DET,Oct 18 7:00 PM,2017,90-102,202.0,-2.5
3,BKN @ IND,Oct 18 7:00 PM,2017,131-140,216.5,-3
4,MIA @ ORL,Oct 18 7:00 PM,2017,109-116,205.5,3.5
...,...,...,...,...,...,...
6,ORL @ ATL,Feb 25 7:00 PM,2023,92-109,223.0,1
7,DEN @ GSW,Feb 25 7:00 PM,2023,119-103,232.5,1
8,OKC @ HOU,Feb 25 7:00 PM,2023,123-110,233.0,6.5
9,CHI @ NOP,Feb 25 7:00 PM,2023,114-106,224.5,-5


In [239]:
table_text = table_text.drop_duplicates()

table_text

,Game,Date,Year,Score,Over/Under,Home Line Spread
0,BOS @ CLE,Oct 17 8:00 PM,2017,99-102,216.0,-4.5
1,HOU @ GSW,Oct 17 10:30 PM,2017,122-121,231.0,-9.5
2,CHA @ DET,Oct 18 7:00 PM,2017,90-102,202.0,-2.5
3,BKN @ IND,Oct 18 7:00 PM,2017,131-140,216.5,-3
4,MIA @ ORL,Oct 18 7:00 PM,2017,109-116,205.5,3.5
...,...,...,...,...,...,...
6,ORL @ ATL,Feb 25 7:00 PM,2023,92-109,223.0,1
7,DEN @ GSW,Feb 25 7:00 PM,2023,119-103,232.5,1
8,OKC @ HOU,Feb 25 7:00 PM,2023,123-110,233.0,6.5
9,CHI @ NOP,Feb 25 7:00 PM,2023,114-106,224.5,-5


In [352]:
# Function to parse game matchup
def parse_matchup(game_str):
    teams = game_str.split(' @ ')
    return teams[0], teams[1], f"{teams[0]} @ {teams[1]}", f"{teams[1]} vs. {teams[0]}"

# Function to convert date format
def convert_date(date_str, year_str):
    try:
        return pd.to_datetime(date_str + " " + year_str, format='%b %d %I:%M %p %Y').strftime('%Y-%m-%d')
    except ValueError:
        print(date_str, year_str)
        
def parse_dataframe(df):
    parsed_data = []

    for index, row in df.iterrows():
        home_team, away_team, matchup, reverse_matchup = parse_matchup(row['Game'])
        game_date = convert_date(row['Date'], row['Year'])
        home_score, away_score = map(int, row['Score'].split('-'))
        parsed_data.append((home_team, game_date, matchup, home_score, away_score, row['Over/Under'], float(row['Home Line Spread'])))
        parsed_data.append((away_team, game_date, reverse_matchup, away_score, home_score, row['Over/Under'], -1 * float(row['Home Line Spread'])))

    parsed_df = pd.DataFrame(parsed_data, columns=["TEAM_NAME", "GAME_DATE", "MATCHUP", "HOME_SCORE", "AWAY_SCORE", "OVER/UNDER", "SPREAD"])
    return parsed_df

In [321]:
def parse_dataframe(df):
    
    parsed_dataframe = pd.DataFrame(columns=["TEAM_NAME", "GAME_DATE", "MATCHUP", "HOME_SCORE", "AWAY_SCORE", "OVER/UNDER", "SPREAD"])
    for i, text in enumerate(df["Game"]):
        if i % 50 == 0: print(f"Row {i}\n {i/len(df) * 100}% Complete")
        parsed_matchup = parse_matchup(text)
        converted_date = convert_date(df["Date"].iloc[i], df["Year"].iloc[i])
        home_score = df["Score"].iloc[i].split("-")[0]
        away_score = df["Score"].iloc[i].split("-")[1]
        ous = df["Over/Under"].iloc[i]
        hls = df["Home Line Spread"].iloc[i]
        als = -1 * df["Home Line Spread"].iloc[i]
        
        home_team, away_team = {}, {}
        home_team[parsed_matchup[0]] = [converted_date, parsed_matchup[2], home_score, away_score, ous, hls]
        away_team[parsed_matchup[1]] = [converted_date, parsed_matchup[3], home_score, away_score, ous, als]
        
        parsed_dataframe = pd.concat([parsed_dataframe, pd.DataFrame(home_team), pd.DataFrame(away_team)])
        
    return parsed_dataframe
            

In [353]:
parsed_dataframe = parse_dataframe(table_text)
parsed_dataframe.head()

Feb 29 5:00 PM 2019
Feb 29 7:30 PM 2019
Feb 29 7:30 PM 2019
Feb 29 8:00 PM 2019
Feb 29 8:00 PM 2019
Feb 29 8:30 PM 2019
Feb 29 8:30 PM 2019
Feb 29 9:00 PM 2019


,TEAM_NAME,GAME_DATE,MATCHUP,HOME_SCORE,AWAY_SCORE,OVER/UNDER,SPREAD
0,BOS,2017-10-17,BOS @ CLE,99,102,216.0,-4.5
1,CLE,2017-10-17,CLE vs. BOS,102,99,216.0,4.5
2,HOU,2017-10-17,HOU @ GSW,122,121,231.0,-9.5
3,GSW,2017-10-17,GSW vs. HOU,121,122,231.0,9.5
4,CHA,2017-10-18,CHA @ DET,90,102,202.0,-2.5


In [354]:
parsed_dataframe.to_csv("ParsedBettingLines.csv")